### Steps to reproduce: <br>
1) Install packages <br>
2) Restart the notebook so packages can turn on properly <br>
3) Run installation again <br>
4) Add the file with combined interview texts ('combined.docx') and dataframe with lemmatized documents ('interview_lemmatized.xlsx') <br>
5) Click preprocessing and look at df of words frequencies and decide which words you want to remove. Add them to 'additional_stopwords' list in preprocessing block and run the cell again <br>
5) Run Topic model <br>
6) Choose the mode and run fit <br>

## Installation of packages

In [1]:
!pip install sklearn==1.0.2
!pip install keras==2.8.0
!pip install tensorflow==2.8.0
!pip install clusteval
!pip install python-docx
!pip install stop-words
!pip install -U sentence-transformers
!pip install chainer
!pip install pyLDAvis
!pip install -U pandas
!pip install -U openpyxl
!pip install hdbscan
!pip install numba==0.51.2
!pip install umap-learn==0.5.2
!pip install numpy==1.21.5

ERROR: Could not find a version that satisfies the requirement sklearn==1.0.2 (from versions: 0.0)
ERROR: No matching distribution found for sklearn==1.0.2
Requirement already up-to-date: sentence-transformers in /home/likich/.local/lib/python3.8/site-packages (2.2.0)
Requirement already up-to-date: pandas in /home/likich/.local/lib/python3.8/site-packages (1.4.1)
Requirement already up-to-date: openpyxl in /home/likich/.local/lib/python3.8/site-packages (3.0.9)


In [2]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import numpy as np
import nltk
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import gensim
from IPython.display import display, HTML
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
import gensim
from datetime import timedelta
import json
import itertools
import numpy as np
import stop_words
from gensim.models import CoherenceModel
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import time
import nltk.data
import docx
from gensim import corpora
from gensim.models import CoherenceModel
from tqdm.notebook import tqdm 
import keras
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV
import hdbscan
from sklearn.metrics import make_scorer
import umap.umap_ as umap
import json
import ast

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-02-20 16:51:32.020601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-20 16:51:32.020651: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to /home/likich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/likich/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/likich/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import numpy
import stop_words
rus = stop_words.get_stop_words('russian')
en = stop_words.get_stop_words('english')
all_sw = rus + en
additional = ['инт', 'инф']
all_sw += additional

In [4]:
import warnings
warnings.filterwarnings('ignore')

## After downloading packages, restart

## Preprocessing

In [5]:
def preprocess(text_file, lemmatized_excel_file, length_restrict, bigram_mincount, additional_stopwords):
  ''' length_restrict - the minimum length of the word to leave in the text
      bigram_mincount – Ignore all words and bigrams with total collected count lower than this value.'''

  print('Reading your transcripts...')

  def getText(text_file):
    doc = docx.Document(text_file)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

  def dataset_raw(text_file):
    text = getText(text_file)
    sentences = []
    tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
    sentences = tokenizer.tokenize(text)
    paragraphs = text.split('\n')
    df = pd.DataFrame(paragraphs,
                    columns = ['paragraphs'],
                    index = range(1, len(paragraphs)+1))
    return df

  df_raw = dataset_raw('example_all.docx')
  print('Raw dataset ready')
  #Provide excel file with lemmatized sentences
  print('Processing your lemmatized dataset...')

  df = pd.read_excel(lemmatized_excel_file, engine="openpyxl", index_col = 0)

  def text_to_array(length_restrict, lemmatized_df):
    ''' length_restrict - the minimum length of the word to leave in the text'''
    x_rus = []
    for i in range(len(lemmatized_df)):  
      string_spl = str(lemmatized_df['paragraphs'].iloc[i]).split()
      for i in string_spl:
        if i == 'nan' or len(i) < length_restrict+1:
          string_spl.remove(i)
      if len(string_spl) > length_restrict:
        x_rus.append(string_spl)
    united =  []
    for i in x_rus:
      for j in i:
        united.append(j)
    df_counts = pd.DataFrame({'text':united})
    df_counts = (df_counts['text'].str.split(expand=True)
                  .stack()
                  .value_counts()
                  .rename_axis('vals')
                  .reset_index(name='count'))
    
    return x_rus, df_counts

  x_rus, df_counts = text_to_array(length_restrict, df)


  def purification(additional_stopwords, array_to_clear, stop_words):
    stop_words += additional_stopwords
    x_rus_c = []
    for i in array_to_clear:
      for j in i:
        if j in stop_words:
          i.remove(j)
    for i in array_to_clear:
      x_rus_c.append(list(set(i)))
    return x_rus_c

  def make_corpus(clear_text_set, bigram_mincount):
    '''bigram_mincount – Ignore all words and bigrams with total collected count lower than this value.'''
    bigram = gensim.models.Phrases(clear_text_set, min_count=bigram_mincount, threshold=40)
    clear_text_set = [bigram[line] for line in clear_text_set]
    x_train_rus = [' '.join(i) for i in clear_text_set]
    dictionary = corpora.Dictionary(clear_text_set)
    corpus = [dictionary.doc2bow(text) for text in clear_text_set]
    return x_train_rus, dictionary, corpus

  print('Purifying the dataset with additional stop words...')
  x_rus_c = purification(additional_stopwords, x_rus, all_sw)
  print('Constructing the corpus...')
  x_train_rus, dictionary, corpus = make_corpus(x_rus_c, bigram_mincount)

  united =  []
  for i in x_rus:
    for j in i:
      united.append(j)
  df_counts_new = pd.DataFrame({'text':united})
  df_counts_new = (df_counts_new['text'].str.split(expand=True)
                  .stack()
                  .value_counts()
                  .rename_axis('vals')
                  .reset_index(name='count'))
  
  return df_raw, df_counts, df_counts_new, x_train_rus, x_rus, dictionary, corpus
      


In [7]:
additional_stopwords = ['понимать', 'думать', 'сделать', 'вообще', 'наверное', 'ничто', 'думать', 'полагать', 'вопрос', 'сделать',
              'знать', 'свой', 'например', 'все', 'таки', 'считать', 'самый', 'поэтому', 'происходить', 'вещь', 'бывать',
              'спрашивать', 'тип', 'насколько', 'точка', 'зрение', 'якобы', 'инт', 'инф', 'либо', 'целое', 'далее', 'скажем']
all_sw += additional_stopwords

# additional_stopwords = []

df_raw, df_counts, df_counts_new, x_train_rus, x_rus, dictionary, corpus = preprocess('combined.docx', 'interview_lemmatized.xlsx', 2, 3, additional_stopwords)

Reading your transcripts...
Raw dataset ready
Processing your lemmatized dataset...
Purifying the dataset with additional stop words...
Constructing the corpus...


## df with frequencies

In [101]:
df_counts_new[0:50]

,vals,count
0,работа,162
1,работать,86
2,хороший,81
3,говорить,78
4,делать,72
5,момент,71
6,решение,69
7,пойти,64
8,должный,60
9,армия,59


In [8]:
import pickle
dictionary.save('dictionary')
corpora.MmCorpus.serialize('corpus',corpus)
with open("x_train_rus", "wb") as fp:   #Pickling
    pickle.dump(x_train_rus, fp)
with open("x_rus", "wb") as fp:   #Pickling
    pickle.dump(x_rus, fp)
with open("all_sw", "wb") as fp:   #Pickling
    pickle.dump(all_sw, fp)

## Fitting

BERT_HDBSCAN CHECK

Remember that hdbscan can't precisely make the model with the set number of topics

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from TM import Topic_Model

In [13]:
tm = Topic_Model(k = 10, method = 'BERT')

Initialized


In [14]:
model_res = tm.fit(corpus, dictionary, cluster_model='hdbscan')

Getting vector representations for BERT ...


  0%|          | 0/1343 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
_________________________________________________________________
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possi

## Graph

In [15]:
nodes = []
links = []
for key, values in model_res.items():
  if key == -1:
    pass
  else:
    for tupl in values:
      nodes.append({'name': tupl[0]})
    # for tupl1 in range(len(values)-1):
    #   links.append({'source':0, 'target': tupl1})
nodes_str = []
for i in nodes:
  nodes_str.append(str(i))
nodes_str = list(set(nodes_str))
clear_nodes = []
for i in nodes_str:
  clear_nodes.append(ast.literal_eval(i))
node_namings_only = []
for node in clear_nodes:
  for key, node_name in node.items():
    node_namings_only.append(node_name)
links = []
for node in clear_nodes:
  for key, node_name in node.items():
    #print(node_name)
    for k,v in model_res.items():
      if k == -1:
        pass
      else:
        for tupl_id in range(len(v)):
          if  v[0][0] == node_name:
            links.append( {"source":node_namings_only.index(v[0][0]), 'target': node_namings_only.index(v[tupl_id][0])})         
#убираем одинаковые сурс и таргет
clear_links = []
for i in range(len(links)):
  compare = []
  for k,v in links[i].items():
    compare.append(v)
  if compare[0] == compare[1]:
    pass
  else:
    clear_links.append(links[i])

In [16]:
dict_json = {'links': clear_links, 'nodes': clear_nodes}
with open('graph.json', 'w', encoding='utf-8') as file:
    json.dump(dict_json, file, ensure_ascii=False)

LDA CHECK

In [9]:
tm1 = Topic_Model(k = 10, method = 'LDA')

Initialized


In [10]:
model_res, to_show = tm1.fit(corpus, dictionary)

Fitting LDA ...
Fitting LDA Done!
Topic: 0 Word: 0.014*"образование" + 0.014*"большой" + 0.014*"получать" + 0.011*"работа" + 0.010*"говорить" + 0.009*"помощь" + 0.009*"учиться" + 0.008*"пойти" + 0.008*"детство" + 0.008*"сторона"
Topic: 1 Word: 0.028*"говориться" + 0.021*"принцип" + 0.013*"барьер" + 0.012*"итог" + 0.010*"допускать" + 0.009*"неплохо" + 0.009*"извинять" + 0.009*"вино" + 0.009*"вроде" + 0.009*"оставаться"
Topic: 2 Word: 0.012*"пример" + 0.011*"естественно" + 0.010*"мнение" + 0.010*"работа" + 0.010*"начинать" + 0.009*"решать" + 0.009*"должный" + 0.009*"обязательно" + 0.008*"мама" + 0.007*"становиться"
Topic: 3 Word: 0.015*"работать" + 0.011*"хороший" + 0.010*"получать" + 0.009*"стараться" + 0.009*"приводить" + 0.008*"родитель" + 0.008*"разница" + 0.008*"чувствовать" + 0.008*"выбор" + 0.008*"некоторый"
Topic: 4 Word: 0.017*"делать" + 0.016*"стыд" + 0.014*"естественно" + 0.013*"совет" + 0.013*"вина" + 0.012*"чувство" + 0.012*"основной" + 0.011*"виноватый" + 0.011*"дело" + 0.0

/home/likich/TM_graph/TM.py:244: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  processed_docs = np.array(x_train_rus_clear)


C_v coherence:  0.396146479905337
U_mass coherence:  -12.06533472955722
UCI :  -11.171977147541963
NPMI :  -0.3611699559384644
Topic_diversity : 0.85
Kullback-Leibler Divergence : 0.8884020006417157


In [12]:
import ast

In [13]:
nodes = []
links = []
for topic in model_res:
    for word in topic:
      nodes.append({'name': word})
nodes_str = []
for i in nodes:
  nodes_str.append(str(i))
nodes_str = list(set(nodes_str))
clear_nodes = []
for i in nodes_str:
  clear_nodes.append(ast.literal_eval(i))
node_namings_only = []
for node in clear_nodes:
  for key, node_name in node.items():
    node_namings_only.append(node_name)

for node in clear_nodes:
  for key, node_name in node.items():
    #print(node_name)
    for topic in model_res:
        for word_id in range(len(topic)):
          if  topic[0] == node_name:
            links.append( {"source":node_namings_only.index(topic[0]), 'target': node_namings_only.index(topic[word_id])})         
#убираем одинаковые сурс и таргет
clear_links = []
for i in range(len(links)):
  compare = []
  for k,v in links[i].items():
    compare.append(v)
  if compare[0] == compare[1]:
    pass
  else:
    clear_links.append(links[i])

In [14]:
dict_json = {'links': clear_links, 'nodes': clear_nodes}
with open('graph.json', 'w', encoding='utf-8') as file:
    json.dump(dict_json, file, ensure_ascii=False)

BERT_KMEANS CHECK

In [ ]:
tm2 = Topic_Model(k = 10, method = 'BERT')
tm2.fit(corpus, dictionary, cluster_model='Kmeans')

Initialized
Clustering embeddings ...
Getting vector representations for BERT ...


  0%|          | 0/1302 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
Clustering embeddings. Done!
Getting topic words
_________________________________________________________________
C_v coherence:  0.4029325251696475
U_mass coherence:  -3.074523991532285
UCI :  -0.828621006136968
NPMI :  -0.004317247693360652
Topic_diversity :  0.42
_________________________________________________________________


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,использовать,алиса,дом,колонка,ассистент,находиться,семья,включать,пользоваться,комната
Topic 1,музыка,включать,говорить,алиса,использовать,колонка,слушать,запрос,получаться,ребенок
Topic 2,включать,музыка,алиса,говорить,колонка,песня,либо,начинать,выключать,слушать
Topic 3,использовать,устройство,запрос,колонка,музыка,включать,ассистент,алиса,использование,ситуация
Topic 4,включать,вечер,утро,музыка,алиса,использовать,погода,свет,ужин,завтрак
Topic 5,ассистент,голосовой,говорить,использовать,телефон,колонка,запрос,музыка,целое,идеальный
Topic 6,использовать,дома,говорить,ассистент,устройство,самостоятельно,пользоваться,колонка,голосовой,музыка
Topic 7,ассистент,голосовой,использовать,говорить,запрос,колонка,телефон,алиса,начинать,пользоваться
Topic 8,проблема,говорить,сталкиваться,запрос,использование,ситуация,трудность,ассистент,недостаток,устройство
Topic 9,говорить,целое,скоро,колонка,ассистент,использовать,умный,пользоваться,итог,включать


BERT_LDA_KMEANS CHECK

In [ ]:
tm3 = Topic_Model(k = 10, method = 'BERT_LDA')
tm3.fit(corpus, dictionary, cluster_model='Kmeans')

Initialized
Clustering embeddings ...
Getting vector representations
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
Getting vector representations for BERT ...


  0%|          | 0/1302 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
Fitting Autoencoder ...
Fitting Autoencoder Done!
Getting vector representations. Done!
Clustering embeddings. Done!
Getting topic words
_________________________________________________________________
C_v coherence:  0.4801940481979917
U_mass coherence:  -2.7254009025118493
UCI :  -0.5351680504703986
NPMI :  0.002609492583029728
Topic_diversity :  0.44
_________________________________________________________________


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,запрос,музыка,колонка,включать,поболтать,пробовать,реакция,говорить,погода,соответственно
Topic 1,запрос,колонка,ребенок,общий,ассистент,делать,использовать,целое,говорить,музыка
Topic 2,использовать,колонка,ассистент,подробно,сталкиваться,говорить,рассказывать,устройство,использование,начинать
Topic 3,говорить,момент,голос,включать,колонка,телефон,дело,музыка,находить,прямо
Topic 4,ассистент,колонка,использование,голосовой,умный,телефон,говорить,целое,алиса,дом
Topic 5,музыка,включать,алиса,говорить,колонка,рука,делать,ситуация,голос,послушать
Topic 6,использовать,ассистент,голосовой,пользоваться,колонка,алиса,телефон,дома,устройство,говорить
Topic 7,включать,говорить,музыка,алиса,запрос,просить,распознавать,либо,ситуация,реагировать
Topic 8,алиса,включать,музыка,говорить,запрос,разговаривать,погода,начинать,использовать,редко
Topic 9,включать,музыка,использовать,алиса,говорить,колонка,песня,дело,запрос,просить


BERT_LDA_HDBSCAN CHECK

In [ ]:
tm4 = Topic_Model(k = 10, method = 'BERT_LDA')
tm4.fit(corpus, dictionary, cluster_model='hdbscan')

Initialized
Getting vector representations
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
Getting vector representations for BERT ...


  0%|          | 0/1302 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
Fitting Autoencoder ...
Fitting Autoencoder Done!
Getting vector representations. Done!
_________________________________________________________________
C_v coherence:  0.4476630284796845
U_mass coherence:  -3.827216020450319
UCI :  -1.4743960067842226
NPMI :  -0.012532944132612864
Topic_diversity :  0.7625
_________________________________________________________________


{-1: [('включать', 0.0340019151379548),
  ('музыка', 0.03280170828158372),
  ('говорить', 0.0273588715704079),
  ('колонка', 0.026158221774226977),
  ('запрос', 0.024661060076250973),
  ('алиса', 0.021672852078464193),
  ('использовать', 0.019399672521942902),
  ('слушать', 0.01714579522916201),
  ('делать', 0.016563958323650336),
  ('получаться', 0.016165861360786207),
  ('либо', 0.01608898274450433),
  ('основной', 0.015869491518602882),
  ('ассистент', 0.01496984377686056),
  ('допускать', 0.014920369793391952),
  ('просить', 0.014844846963177064),
  ('проблема', 0.014761337099280373),
  ('ребенок', 0.014610586577831248),
  ('нравиться', 0.014211792374004908),
  ('слышать', 0.013916681283651934),
  ('начинать', 0.01370292696184182)],
 0: [('подробно', 0.12001039248603587),
  ('ассистент', 0.09413494286182006),
  ('использовать', 0.08478269341082992),
  ('устройство', 0.07371394503906055),
  ('проживать', 0.07118372016428363),
  ('возможность', 0.06778157338855849),
  ('колонка', 0.0

## Черновик метрик

In [ ]:
ldamodel = gensim.models.LdaMulticore(corpus, num_topics=10, 
                                       id2word = dictionary,
                                       workers = 2, passes=10,
                                       random_state=100,
                                       chunksize=100)

In [ ]:
term_topic = ldamodel.get_topics()

In [ ]:
term_topic.shape

(10, 2624)

In [ ]:
def _LOR(P, Q):
    lor = 0
    for v, w in zip(P, Q):
        if v > 0 or w > 0:
            lor = lor + np.abs(np.log(v) - np.log(w))
    return lor / len(P)
def Kullback_Leibler(ldamodel):
    beta = ldamodel.get_topics()
    kl_div = 0
    count = 0
    for i, j in itertools.combinations(range(len(beta)), 2):
        kl_div += _LOR(beta[i], beta[j])
        count += 1
    return kl_div / count

In [ ]:
Kullback_Leibler(ldamodel)

0.9005306353292816